In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.AdvancedSolvers;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
var DB = OpenOrCreateDatabase(@"W:\work\scratch\jw52xeqa\DB_rotSphere_CoreScaling"); DB.Sessions

#0: strong_scaling_3	J17496_k4_t1_Stokes_c64_Re50_mue_1*	09/21/2021 00:01:57	5170e026...
#1: strong_scaling_3	J17496_k4_t1_Stokes_c32_Re50_mue_1*	09/21/2021 00:01:46	a5ce7096...
#2: strong_scaling_3	J17496_k4_t1_Stokes_c8_Re50_mue_1*	09/21/2021 00:01:26	297ec062...
#3: strong_scaling_3	J17496_k4_t1_Stokes_c16_Re50_mue_1*	09/21/2021 00:01:33	b1769cb6...
#4: strong_test_2	J17496_k4_t1_Stokes_c128_Re50_mue_1	09/20/2021 22:04:10	d2001bdd...
#5: strong_test_2	J17496_k4_t1_Stokes_c128_Re50_mue_1_20Iter*	09/20/2021 21:24:25	f9dc0327...
#6: strong_test_2	J17496_k4_t1_Stokes_c128_Re50_mue_1_c128_Re50_mue_1_no_reortho	09/20/2021 17:13:15	f7686f7f...
#7: strong_test_2	J17496_k4_t1_Stokes_c128_Re50_mue_1	09/20/2021 16:46:16	5dab4999...
#8: strong_test_2	J17496_k4_t1_Stokes_c128_Re50_mue_1_c128_Re50_mue_1_2	09/20/2021 16:48:24	975fc7a8...
#9: strong_test_2	J17496_k4_t1_Stokes_c128_Re50_mue_1_c128_Re50_mue_1_no_reortho	09/20/2021 16:46:38	242a3de4...
#10: strong_test_2	J17496_k4_t1_Stokes_c128_Re50_

In [ ]:
//DB.Sessions.Where(s=>Convert.ToInt32(s.KeysAndQueries["DGdegree:Velocity*"])==3).Where(s=>s.ProjectName=="strong_scaling")
DB.Sessions

In [ ]:
//var SomeSessions = new ISessionInfo[]{DB.Sessions.Where(s=>s.NumberOfCores()==128).Pick(2,3,4)};
var SomeSessions = new ISessionInfo[]{DB.Sessions.Where(s=>s.NumberOfCores()==128).Pick(20)};

In [ ]:
static var PreComputeProfiling = new Dictionary<System.Guid,MethodCallRecord[]>();
foreach(var sess in SomeSessions){
    MethodCallRecord[] mcrs = null;
    try{
        mcrs = sess.GetProfiling();
    } catch (Exception ex){
        Console.WriteLine("Name:"+sess.Name);
        Console.WriteLine(ex.Message);
    }
    PreComputeProfiling.Add(sess.ID, mcrs);
}

In [ ]:
static class AddCols {
    static public MethodCallRecord[] GetMCR(ISessionInfo SI){
        MethodCallRecord[] mcrs = null;
        PreComputeProfiling.TryGetValue(SI.ID,out mcrs);
        return mcrs;
    }
    
    static private double TimeOnProc(MethodCallRecord mcr, string method){
        double time = 0;
        foreach(var child in mcr.FindChildren(method)){
            time += child.TimeSpentInMethod.TotalSeconds;
            //time += child.TimeSpentInMPIBlocking.TotalSeconds;
        }
        /*
        var child = mcr.FindChildren(method).Pick(0);
        time = child.TimeExclusive.TotalSeconds;
        */
        /*
        double time = -1;
        foreach(var child in mcr.FindChildren(method)){
            var childtime = child.TimeSpentInMethod.TotalSeconds;
            //var childtime = child.TimeSpentInMPIBlocking.TotalSeconds;
            time = childtime > time? childtime:time;
        }
        */
        return time;
    }
    
    static public double AverageTime(ISessionInfo SI,string method){
        var mcrs       = GetMCR(SI);
        double avtime = 0;

        foreach(var mcr in mcrs){
            avtime+=TimeOnProc(mcr,method);
        }
        avtime = avtime / mcrs.Count();
        return avtime;
    }

    static public double MaxTime(ISessionInfo SI,string method){
        var mcrs       = GetMCR(SI);
        double maxtime = -1;

        foreach(var mcr in mcrs){
            double time =TimeOnProc(mcr,method);    
            maxtime = time>maxtime?time:maxtime;
        }
        return maxtime;
    }

    static public object LSolverIter(ISessionInfo SI){
        var mcr          = GetMCR(SI)[0];
        string searchstr = "BoSSS.Solution.AdvancedSolvers.DirectSolver.Solve";
        var ndS = mcr.FindChildren(searchstr);
        var nd  = ndS.ElementAt(0);
        return nd.CallCount;
    }

    static public object NoOfCores(ISessionInfo SI){
        return SI.ComputeNodeNames.Count();
    }

    static public object NoOfNodes(ISessionInfo SI){
        return SI.ComputeNodeNames.GroupBy(s=>s).Count();
    }
    
    static public object MGDepth(ISessionInfo SI){
        var mcr = GetMCR(SI)[0];
        double NInit  = mcr.FindChildren("BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Init").ToArray().Length;
        return NInit + 1;
    }
}

In [ ]:
//SomeSessions.Select(s=>AddCols.MGDepth(s))
//SomeSessions.Pick(0).KeysAndQueries.Keys.Contains("DGdegree:Velocity*")
SomeSessions.Select(s=>AddCols.LSolverIter(s))

index,value
0,92


In [ ]:
var Sess = SomeSessions.Pick(1);
PreComputeProfiling.TryGetValue(Sess.ID,out MethodCallRecord[] mcrs);
mcrs[0].FindChildren("Slv Iter").Pick(0).TimeSpentInMethod.TotalSeconds

Error: System.ArgumentOutOfRangeException: Index was out of range. Must be non-negative and less than the size of the collection. (Parameter 'index')
   at System.SZArrayHelper.get_Item[T](Int32 index)
   at System.Linq.Enumerable.ElementAt[TSource](IEnumerable`1 source, Int32 index)
   at Submission#72.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
string[] Functraces = new string[]{               
   "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.AddSol",
   //"BoSSS.Solution.AdvancedSolvers.Schwarz.Solve",
   "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.MinimizeResidual",
   "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Residual",
   //"BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.Solve",
   "overlap_scaling",
   "block_solve_level",
   "caching",
   "nocaching",
   "BoSSS.Solution.AdvancedSolvers.OrthonormalizationMultigrid.VerivyCurrentResidual",
   "Restriction",
   "Prolongation",
   "Slv Iter"
};

In [ ]:
var SesTab = new System.Data.DataTable();
var Col         = SesTab.Columns.Add("NoCores", typeof(Int32));  
Col.AllowDBNull = false;
SesTab.Columns.Add("Iter", typeof(Int32)); 
SesTab.Columns.Add("DGdegree", typeof(Int32));
SesTab.Columns.Add("Name",typeof(String));
var ColDic = new Dictionary<string, string>();
foreach(string Ftrace in Functraces){
    string[] Schnipsel = Ftrace.Split('.');
    int L = Schnipsel.Length;
    string ColumnName = "";
    if(L>1)
        ColumnName = String.Concat(Schnipsel[L-2],'.',Schnipsel[L-1]);
    else
        ColumnName = Schnipsel.Last();
    SesTab.Columns.Add(ColumnName,typeof(Double));
    ColDic.Add(ColumnName,Ftrace);
} SesTab.GetColumnNames()

index,value
0,NoCores
1,Iter
2,DGdegree
3,Name
4,OrthonormalizationMultigrid.AddSol
5,OrthonormalizationMultigrid.MinimizeResidual
6,OrthonormalizationMultigrid.Residual
7,overlap_scaling
8,block_solve_level
9,caching


In [ ]:
foreach(var sess in SomeSessions){
    var newrow = SesTab.NewRow();
    foreach(var col in SesTab.Columns){
        string cName = (col as System.Data.DataColumn).ColumnName;
        ColDic.TryGetValue(cName, out string Ftrace);
        try{
            newrow[cName]=AddCols.MaxTime(sess,Ftrace);
        } catch (Exception ex){
            //Die anderen Columns halt ...
        } 
    }
    newrow["NoCores"] = Convert.ToInt32(AddCols.NoOfCores(sess));
    newrow["Iter"] = Convert.ToInt32(AddCols.LSolverIter(sess));
    newrow["DGdegree"] = Convert.ToInt32(sess.KeysAndQueries["DGdegree:Velocity*"]);
    newrow["Name"] = Convert.ToString(sess.Name);
    SesTab.Rows.Add(newrow);
} SesTab.Print()

   NoCores Iter DGdegree Name                                 OrthonormalizationMultigrid.AddSol OrthonormalizationMultigrid.MinimizeResidual OrthonormalizationMultigrid.Residual overlap_scaling    block_solve_level caching nocaching OrthonormalizationMultigrid.VerivyCurrentResidual Restriction Prolongation Slv Iter    
0: 128     92   4        J17496_k4_t1_Stokes__c128_Re50_mue_1 203.2427153                        1.2685959                                    3.4500699999999997                   332.73627700000003 620.1683336       0       0         0                                                 0           0            2558.434473 

In [ ]:
var theDict = new List<Tuple<string,DashTypes,LineColors>>();
public static Tuple<string,DashTypes,LineColors>[] AofFormats;
var dashes = new DashTypes[]{DashTypes.Dashed,DashTypes.DotDashed,DashTypes.DotDotDashed};
var points= new PointTypes[]{PointTypes.OpenCircle,PointTypes.OpenCircle,PointTypes.OpenDiamond,PointTypes.OpenUpperTriangle};
var colors = new LineColors[]{LineColors.Red, LineColors.Green, LineColors.Blue, LineColors.Magenta, LineColors.Yellow, LineColors.Black, LineColors.Orange, LineColors.Grey};

int iSweep=0;
foreach(string Name in SesTab.GetColumnNames()){
    //if(Name!="OrthonormalizationMultigrid.MinimizeResidual"&&Name!="overlap_scaling")
    //    continue;
    if(Name=="NoCores"||Name=="Iter"||Name=="DGdegree")
        continue;
    int one=iSweep%(dashes.Length-1);
    int two=iSweep/(dashes.Length-1);
    theDict.Add(new Tuple<string,DashTypes,LineColors>(Name,dashes[one],colors[two]));
    iSweep++;
}
AofFormats = theDict.ToArray()

index,Item1,Item2,Item3
0,Name,Dashed,Red
1,OrthonormalizationMultigrid.AddSol,DotDashed,Red
2,OrthonormalizationMultigrid.MinimizeResidual,Dashed,Green
3,OrthonormalizationMultigrid.Residual,DotDashed,Green
4,overlap_scaling,Dashed,Blue
5,block_solve_level,DotDashed,Blue
6,caching,Dashed,Magenta
7,nocaching,DotDashed,Magenta
8,OrthonormalizationMultigrid.VerivyCurrentResidual,Dashed,Yellow
9,Restriction,DotDashed,Yellow


In [ ]:
string thisdir = Directory.GetCurrentDirectory();
SesTab.ToCSVFile(thisdir+@"\table.csv"); thisdir

b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\strongScaling\IBM_RotCube

In [ ]:
public static class DoWhatUWant{
    public static PlotRowSelectorEx Solver_DG_Proj(int DG){
        return delegate (int iSweep, int iTabRow, 
                     IDictionary<string, object> Row, 
                     out string Nmn, 
                     out PlotFormat Fmt,
                     out double xValue, out double yValue) { 

               int k  = Convert.ToInt32(Row["DGdegree"]);

               if(k != DG ) {
                    // degree does not match -> not in this plot
                    Nmn    = null;
                    Fmt    = null;
                    xValue = 0;
                    yValue = 0;
                    return;
               } 
               
               Fmt           = new PlotFormat();
               Fmt.PointSize = 0.5;
               Fmt.Style     = Styles.LinesPoints;
               Fmt.LineWidth = 3;

               Nmn = AofFormats[iSweep].Item1;
               Fmt.DashType = AofFormats[iSweep].Item2;
               Fmt.LineColor = AofFormats[iSweep].Item3;
               double iter = Convert.ToDouble(Row["Iter"]);
               yValue = Convert.ToDouble(Row[Nmn])/iter;
               xValue    = Convert.ToDouble(Row["NoCores"]);
               
           };
    }

    public static PlotRowSelectorEx Solver_DG_Proj(){
        return delegate (int iSweep, int iTabRow, 
                     IDictionary<string, object> Row, 
                     out string Nmn, 
                     out PlotFormat Fmt,
                     out double xValue, out double yValue) { 

               int k           = Convert.ToInt32(Row["DGdegree"]);
               
               Fmt           = new PlotFormat();
               Fmt.PointSize = 0.5;
               Fmt.Style     = Styles.LinesPoints;
               Fmt.LineWidth = 3;
               
               switch(k){
                case 2:
                    Nmn="k2";
                    Fmt.LineColor = LineColors.Red;
                    break;
                case 3:
                    Nmn="k3";
                    Fmt.LineColor = LineColors.Orange;
                    break;
                case 4:
                    Nmn="k4";
                    Fmt.LineColor = LineColors.Blue;
                    break;
                default:
                    throw new Exception("vergiss es");
                }
                double time = Convert.ToDouble(Row["Slv Iter"]);
                double iter = Convert.ToDouble(Row["Iter"]);
                //yValue = time/iter;
                yValue=Convert.ToDouble(Row["Iter"]);
                xValue    = Convert.ToDouble(Row["NoCores"]);
           };
    }

}
    

In [ ]:
int k = 4;
int xMin = 4;
int xMax = 512;

In [ ]:
Plot2Ddata[,] multiplots = new Plot2Ddata[1,1];
multiplots[0,0] = SesTab.ToPlot(DoWhatUWant.Solver_DG_Proj(k),AofFormats.Length);
multiplots[0,0].LogX = true;
multiplots[0,0].LogY = true;
multiplots[0,0].tmargin = 0;
multiplots[0,0].bmargin = 5;
multiplots[0,0].XrangeMin=xMin;
multiplots[0,0].XrangeMax=xMax;
multiplots[0,0].Ylabel = "wallclock time";
multiplots[0,0].Xlabel = "cores";
multiplots[0,0].LegendAlignment=new string[]{"i","b","r"};
multiplots.PlotNow()